In [1]:
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Input, RepeatVector, Dot, Concatenate, LSTM, Bidirectional

Using TensorFlow backend.


### RepeatVector
1. You have a vector, but requires a batch dimension
2. Only 2 dimensional vector can be passed to RepeatVector
3. Added a dimension between batch dimension and original dimension
4. If ignore batch dimension, repeat vector adds one dimension at the begining

In [2]:
dim = 12
vec = np.random.randint(1,10,size = (1, dim))
vec

array([[7, 8, 7, 1, 3, 9, 9, 7, 8, 1, 9, 4]])

In [3]:
inputs = Input(shape = (dim,))
outputs = RepeatVector(3)(inputs)
model = Model(inputs = inputs, outputs = outputs)

In [4]:
model.predict(vec)

W0330 18:18:35.343700 23416 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



array([[[7., 8., 7., 1., 3., 9., 9., 7., 8., 1., 9., 4.],
        [7., 8., 7., 1., 3., 9., 9., 7., 8., 1., 9., 4.],
        [7., 8., 7., 1., 3., 9., 9., 7., 8., 1., 9., 4.]]], dtype=float32)

In [5]:
model.predict(vec).shape

(1, 3, 12)

high dimensional vector rasises an error

In [6]:
# vec = np.random.randint(1,10,size = (1, dim,3))
# model.predict(vec)

## Concatenate

In [7]:
input1 = Input(shape = (dim,))
input2 = Input(shape = (dim,))
outputs = Concatenate(axis = -1)([input1, input2])
model2 = Model([input1, input2], outputs)

In [8]:
input_1_data = np.random.randint(1,10,size = (1, dim))
input_2_data = np.random.randint(1,10,size = (1, dim))
print('Input1: ', input_1_data)
print('Input2: ', input_2_data)
preds = model2.predict([input_1_data, input_2_data])
preds

Input1:  [[6 3 7 3 8 4 7 3 2 2 3 9]]
Input2:  [[4 5 3 3 3 4 3 5 3 4 2 3]]


array([[6., 3., 7., 3., 8., 4., 7., 3., 2., 2., 3., 9., 4., 5., 3., 3.,
        3., 4., 3., 5., 3., 4., 2., 3.]], dtype=float32)

In [9]:
input_1_data.shape

(1, 12)

In [10]:
preds.shape

(1, 24)

### Concate with different lengths

In [11]:
dim1 = 5
dim2 = 7

input1 = Input(shape = (dim1,))
input2 = Input(shape = (dim2,))
outputs = Concatenate(axis = -1)([input1, input2])
model3 = Model([input1, input2], outputs)

In [12]:
input_1_data = np.random.randint(1,10,size = (1, dim1))
input_2_data = np.random.randint(1,10,size = (1, dim2))
model3.predict([input_1_data, input_2_data]).shape

(1, 12)

### Multi-dimensional concatenation

You would never want to concatenate things on 0-th dimension, since it is batch dimension      
You concat things on -1 dimension mostly.        
On high dimensional data, you may concat things on specified dimension       

In [13]:
input1 = Input(shape = (dim1,dim2,))
input2 = Input(shape = (dim1,dim2,))
outputs = Concatenate(axis = 1)([input1, input2])
model4 = Model([input1, input2], outputs)

In [14]:
input_data_1 = np.random.randint(1,10,size = (1, dim1, dim2))
input_data_2 = np.random.randint(1,10,size = (1, dim1, dim2))

In [15]:
input_data_1

array([[[2, 7, 6, 8, 7, 9, 7],
        [8, 6, 1, 7, 3, 4, 9],
        [8, 4, 6, 4, 1, 4, 4],
        [2, 8, 7, 5, 8, 8, 4],
        [7, 8, 5, 5, 3, 9, 4]]])

In [16]:
input_data_2

array([[[2, 5, 2, 4, 8, 1, 6],
        [6, 6, 7, 9, 7, 3, 6],
        [2, 3, 8, 2, 2, 6, 6],
        [2, 8, 9, 8, 8, 7, 7],
        [9, 2, 1, 1, 1, 2, 8]]])

In [17]:
preds = model4.predict([input_data_1, input_data_2])
preds

array([[[2., 7., 6., 8., 7., 9., 7.],
        [8., 6., 1., 7., 3., 4., 9.],
        [8., 4., 6., 4., 1., 4., 4.],
        [2., 8., 7., 5., 8., 8., 4.],
        [7., 8., 5., 5., 3., 9., 4.],
        [2., 5., 2., 4., 8., 1., 6.],
        [6., 6., 7., 9., 7., 3., 6.],
        [2., 3., 8., 2., 2., 6., 6.],
        [2., 8., 9., 8., 8., 7., 7.],
        [9., 2., 1., 1., 1., 2., 8.]]], dtype=float32)

In [18]:
preds.shape

(1, 10, 7)

In [19]:
input1 = Input(shape = (dim1,dim2,))
input2 = Input(shape = (dim1,dim2,))
outputs = Concatenate(axis = 2)([input1, input2])
outputs_1 = Concatenate(axis = -1)([input1, input2])
model5 = Model([input1, input2], outputs)
model6 = Model([input1, input2], outputs_1)

preds = model5.predict([input_data_1, input_data_2])
preds

array([[[2., 7., 6., 8., 7., 9., 7., 2., 5., 2., 4., 8., 1., 6.],
        [8., 6., 1., 7., 3., 4., 9., 6., 6., 7., 9., 7., 3., 6.],
        [8., 4., 6., 4., 1., 4., 4., 2., 3., 8., 2., 2., 6., 6.],
        [2., 8., 7., 5., 8., 8., 4., 2., 8., 9., 8., 8., 7., 7.],
        [7., 8., 5., 5., 3., 9., 4., 9., 2., 1., 1., 1., 2., 8.]]],
      dtype=float32)

In [20]:
preds.shape

(1, 5, 14)

In [21]:
preds = model5.predict([input_data_1, input_data_2])
preds

array([[[2., 7., 6., 8., 7., 9., 7., 2., 5., 2., 4., 8., 1., 6.],
        [8., 6., 1., 7., 3., 4., 9., 6., 6., 7., 9., 7., 3., 6.],
        [8., 4., 6., 4., 1., 4., 4., 2., 3., 8., 2., 2., 6., 6.],
        [2., 8., 7., 5., 8., 8., 4., 2., 8., 9., 8., 8., 7., 7.],
        [7., 8., 5., 5., 3., 9., 4., 9., 2., 1., 1., 1., 2., 8.]]],
      dtype=float32)

In [22]:
preds.shape

(1, 5, 14)

In [23]:
model5.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 5, 7)         0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 5, 7)         0                                            
__________________________________________________________________________________________________
concatenate_4 (Concatenate)     (None, 5, 14)        0           input_8[0][0]                    
                                                                 input_9[0][0]                    
Total params: 0
Trainable params: 0
Non-trainable params: 0
__________________________________________________________________________________________________


In [49]:
mat = np.ones((10,5))*(np.array([0,1,2,3,4,5,6,7,8,9]).reshape(-1,1))
mat = np.array([mat])

In [50]:
vec = np.ones((10,1))*2
vec = np.array([vec])

In [54]:
Mat = Input(shape = (10,5))
Vec = Input(shape =(10,1))
outputs = Dot(axes = 1)([Vec, Mat])
model_d = Model([Mat, Vec], outputs)

In [55]:
model_d.predict([mat,vec])

array([[[90., 90., 90., 90., 90.]]], dtype=float32)

In [56]:
mat

array([[[0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3.],
        [4., 4., 4., 4., 4.],
        [5., 5., 5., 5., 5.],
        [6., 6., 6., 6., 6.],
        [7., 7., 7., 7., 7.],
        [8., 8., 8., 8., 8.],
        [9., 9., 9., 9., 9.]]])

In [57]:
vec

array([[[2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.],
        [2.]]])